In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import json
import os
import accelerate

In [ ]:
model_dir = "/content/drive/MyDrive/Models"

In [ ]:
# model_name = "microsoft/phi-1"
# model_name =  "Salesforce/codegen-350M-mono"
# model_name = "shaddie/rocketry_roqeto_model" #
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_dir = model_dir + "/rocketry-roqeto-model"

In [ ]:
# ====== Load Dataset ======
def load_json_dataset1(data):
    return Dataset.from_list([{
       "text": f"Question: {item['question']}\nAnswer: {item['answer']}"
    } for item in dataset1["train"]["spacesystems"][0:4500] ])

def load_json_dataset2(data):
    return Dataset.from_list([{
       "text": f"Question: {item['question']}\nAnswer: {item['answer']}"
    } for item in dataset2["train"] ])

In [ ]:
dataset1 = load_dataset("shaddie/space_systems_qas_dataset")
data1 = load_json_dataset1(dataset1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data1

Dataset({
    features: ['text'],
    num_rows: 4500
})

In [ ]:
dataset2 = load_dataset("shaddie/rocketry_qas_dataset")
data2 = load_json_dataset2(dataset2)

In [ ]:
data2

Dataset({
    features: ['text'],
    num_rows: 757
})

In [ ]:
dataset = concatenate_datasets([data1, data2])

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 5257
})

In [ ]:
# # Load tokenizer and model
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
# === Load Model & Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if "TinyLlama" in model_name:
  tokenizer.pad_token = tokenizer.eos_token  # Avoid pad_token warnings
elif "codegen-350M-mono" in model_name:
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})


In [ ]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

In [ ]:
# === Prepare for PEFT ===
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 1847296 || all params: 1101895680 || trainable%: 0.16764708615610507


In [ ]:
# Tokenize the dataset
def tokenize(example):
    # print(f'example {example["text"]}')
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/5257 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-4,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    report_to="none"
)

In [ ]:
# ====== Trainer ======
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-19-0170af16dc0f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
  # ====== Train ======
trainer.train()

Step,Training Loss
50,2.459500
100,2.339000
150,2.334300
200,2.339100
250,2.343900
300,2.291300
350,2.299300
400,2.235600
450,2.247000
500,2.214200


TrainOutput(global_step=1645, training_loss=2.1528443774191444, metrics={'train_runtime': 2787.9231, 'train_samples_per_second': 9.428, 'train_steps_per_second': 0.59, 'total_flos': 8.36834739879936e+16, 'train_loss': 2.1528443774191444, 'epoch': 5.0})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("shaddie/rocketry_roqeto_peft_model",
                  use_auth_token=True,
                  commit_message="fine-tuning-for-rocketry-knowledge",
                  private=False)

adapter_model.safetensors:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shaddie/roqeto_peft_model/commit/fb05dd1caad290cbffc4f1096f7fd2c31a7c4132', commit_message='fine-tuning-for-rocketry-knowledge', commit_description='', oid='fb05dd1caad290cbffc4f1096f7fd2c31a7c4132', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shaddie/roqeto_peft_model', endpoint='https://huggingface.co', repo_type='model', repo_id='shaddie/roqeto_peft_model'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("shaddie/roqeto_peft_model",
                  use_auth_token=True,
                  commit_message="tokenizer-fine-tuning-for-rocketry-knowledge",
                  private=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shaddie/roqeto_peft_model/commit/5bfc5cb6b3297049a7745f98b525bc6b5d63c1a3', commit_message='tokenizer-fine-tuning-for-rocketry-knowledge', commit_description='', oid='5bfc5cb6b3297049a7745f98b525bc6b5d63c1a3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shaddie/roqeto_peft_model', endpoint='https://huggingface.co', repo_type='model', repo_id='shaddie/roqeto_peft_model'), pr_revision=None, pr_num=None)

In [ ]:
pipe = pipeline("text-generation", model="shaddie/rocketry_roqeto_peft_model", torch_dtype=torch.bfloat16, device_map="auto")

def query(pipe, question):
  # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
  messages = [
      {
          "role": "system",
          "content": "You are a friendly chatbot who always responds in the style of a pirate",
      },
      {"role": "user", "content": question},
  ]
  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  outputs = pipe(prompt, max_new_tokens=148, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
  return outputs[0]["generated_text"]

adapter_config.json:   0%|          | 0.00/803 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/7.41M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
print(query(pipe, "How would you design or create artificial gravity in a space-ship traveliing in outer space?"))

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How would you design or create artificial gravity in a space-ship traveliing in outer space?</s>
<|assistant|>
 Here are some ways to create or design artificial gravity in a space-ship traveling in outer space:
 
 1. Gravity generators: 
   A gravitational generator is a system that creates artificial gravity. It converts the normal force of gravity into the force of torque. 
   
   A small gravitational generator can be installed on a space-ship to provide the necessary gravity to the crew. It usually consists of a small motor and a small device that converts the normal force of gravity into the force of torque. 
   
   A larger generator can be used to provide more gravity. It is typically installed on the space-ship's main structure. It consists of several small


In [ ]:
print(query(pipe, "Question: Can you describe how you would construct a habitable space station in geosynchronous orbit?"))

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
Question: Can you describe how you would construct a habitable space station in geosynchronous orbit?</s>
<|assistant|>
 Unfortunately, this question is beyond the scope of what I can do for you. However, I can provide you with the general steps and guidance for building a habitable space station in geosynchronous orbit. 
 
 Step 1: Design and Construct the Habitable Space Station 
 
 The first step is to design and construct a habitable space station in geosynchronous orbit. In this case, the planet or satellite is assumed to be Earth, and we are working in the context of Earth orbit. 
 
 Step 2: Determine the Required Mass and Energy Balance 
 
 Once we have a design, we must determine the mass and energy balance. This is done by


In [ ]:
print(query(pipe, "How would you launch a rocket with optimal fuel for reaching space?"))

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How would you launch a rocket with optimal fuel for reaching space?</s>
<|assistant|>
 A rocket is a combination of chemical propulsion system and aerodynamic design to achieve the desired final velocity. The optimal fuel for a specific mission is a function of the mission requirements, the fuel requirements, and the payload mass. 
 
 However, in general, the following steps would be needed to launch a rocket with optimal fuel for reaching space:
 
 1. Calculate the orbital mass of the payload and the fuel requirements. 
 2. Calculate the amount of fuel needed for the rocket. 
 3. Determine the amount of oxidizer and fuel that can be used. 
 4. Calculate the amount of oxidizer and fuel that


In [ ]:
def query_model(prompt, max_new_tokens=196):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(query_model("How would you launch a rocket with optimal fuel for reaching space?"))

How would you launch a rocket with optimal fuel for reaching space?

 Space is a vacuum, so the most efficient way to get to space is to use a rocket with as little fuel as possible. 
 
 The most efficient rocket fuel is hydrogen and oxygen. The most efficient way to convert those two elements into a rocket's propellant is to use a chemical rocket. 
 
 Chemical rockets use a chemical reaction to convert the hydrogen and oxygen into a rocket's propellant. 
 
 The most efficient chemical reaction for rocket propulsion is the 
 
 Hydrogen Peroxide 
 
 This reaction produces a lot of heat, so it's not the most efficient way to get the hydrogen and oxygen. 
 
 Hydrogen Peroxide 
 
 Instead, the most efficient chemical reaction for rocket propulsion is the 
 
 Hydrogen Cy
